In [ ]:
# Homework 7
# M33 Analytical Orbit
# Justin Ugaitafa

In [1]:
# import necessary modules
# numpy provides powerful multi-dimensional arrays to hold and manipulate data
import numpy as np
# matplotlib provides powerful functions for plotting figures
import matplotlib.pyplot as plt
# astropy provides unit system and constants for astronomical calculations
import astropy.units as u
import astropy.constants as const
# import Latex module so we can display the results with symbols
from IPython.display import Latex
%matplotlib inline

# **** import CenterOfMass to determine the COM pos/vel of M33
from CenterOfMass2 import CenterOfMass

# **** import the GalaxyMass to determine the mass of M31 for each component
from GalaxyMass import ComponentMass


MW Halo mass is  1.975 solMass
M31 Halo mass is  1.921 solMass
M33 Halo mass is  0.187 solMass
MW Disk mass is  0.075 solMass
M31 Disk mass is  0.12 solMass
M33 Disk mass is  0.009 solMass
MW Bulge mass is  0.01 solMass
M31 Bulge mass is  0.019 solMass
M33 Bulge mass is  0.0 solMass
MW mass is  2.06 solMass
M31 mass is  2.06 solMass
M33 mass is  0.196 solMass
LG mass is  4.316 solMass
MW Baryon function is  0.041
M31 Baryon function is  0.067
M33 Baryon function is  0.046
LG Baryon function is  0.054


In [121]:
class M33AnalyticOrbit:
    """ Calculate the analytical orbit of M33 around M31 """
    
    def __init__(self, filename):
        """input: filename"""
        ### get the gravitational constant (the value is 4.498502151575286e-06)
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        
        ### **** store the output file name
        self.filename = filename
        
        ### get the current pos/vel of M33 
        # **** create an instance of the  CenterOfMass class for M33
        M33COM = CenterOfMass('M33_000.txt',2)
        delta33 = 0.1
        voldec33 = 4
        
        # **** store the velocity VECTOR of the M33 COM (.value to get rid of units)
        M33Pos = M33COM.COM_P(delta33, voldec33)
        
        # **** store the velocity VECTOR of the M33 COM (.value to get rid of units)
        M33Vel = M33COM.COM_V(M33Pos[0],M33Pos[1],M33Pos[2])
        
        ### get the current pos/vel of M31 
        # **** create an instance of the  CenterOfMass class for M31 
        M31COM = CenterOfMass('M31_000.txt',2)
        delta31 = 0.1
        voldec31 = 2
        
        # **** store the velocity VECTOR of the M31 COM (.value to get rid of units)
        M31Pos = M31COM.COM_P(delta31, voldec31)
        
        # **** store the velocity VECTOR of the M31 COM (.value to get rid of units)
        M31Vel = M31COM.COM_V(M31Pos[0],M31Pos[1],M31Pos[2])
        
        ### store the DIFFERENCE between the vectors posM33 - posM31
        # **** create two VECTORs self.r0 and self.v0 and have them be the
        # relative position and velocity VECTORS of M33
        PosDiff = np.subtract(M33Pos,M31Pos).value
        VelDiff = np.subtract(M33Vel, M31Vel).value
        self.r0 = PosDiff
        self.v0 = VelDiff
        
        ### get the mass of each component in M31 disk
        # **** self.rdisk = scale length (no units)
        self.rdisk = 5
        
        # **** self.Mdisk set with ComponentMass function. Remember to *1e12 to get the right units. Use the 
        #      right ptype
        self.Mdisk = ComponentMass('M31_000.txt',2).value*1e12
        
        ### bulge
        # **** self.rbulge = set scale length (no units)
        self.rbulge = 1
        
        # **** self.Mbulge  set with ComponentMass function. Remember to *1e12 to get the right units Use the 
        #      right ptype
        self.Mbulge = ComponentMass('M31_000.txt',3).value*1e12
        
        # Halo
        # **** self.rhalo = set scale length from HW5 (no units)
        self.rhalo = 62 # radius from MassProfile
        
        # **** self.Mhalo set with ComponentMass function. Remember to *1e12 to get the right units. Use the 
        #      right ptype
        self.Mhalo = ComponentMass('M31_000.txt',1).value*1e12
        
    def HernquistAccel(self, M, r_a, posvec):
        """inputs:
                    M = mass of bulge in solar masses
                    r_a = scale length
                    posvec = position vector
           return:
                    Acceleration vector from a Hernquist potential"""
        ### **** Store the magnitude of the position vector
        posmag = np.sqrt(posvec[0]**2 + posvec[1]**2 + posvec[2]**2)
            
        ### *** Store the Acceleration
        Hernquist = -(self.G*M)/posmag*(r_a + posmag)**2 * posvec # formula from HW instructions
            
        # NOTE: we want an acceleration VECTOR so you need to make sure that in the Hernquist equation you 
        # use  -G*M/(rmag *(ra + rmag)**2) * r --> where the last r is a VECTOR 
        return Hernquist
    
    def MiyamotoNagaiAccel(self, M,rd,posvec):# it is easiest if you take as an input a position VECTOR r
        """inputs:
                    M = mass of M31 disk in solar masses
                    rd = self.rdisk
                    posvec = position vector
           return:
                    Accerleration vactor from a Miyamoto_Nagai profile"""
        zd = self.rdisk/5
            
            ## Acceleration **** follow the formula in the HW instructions
        # AGAIN note that we want a VECTOR to be returned  (see Hernquist instructions)
        # this can be tricky given that the z component is different than in the x or y directions. 
        # we can deal with this by multiplying the whole thing by an extra array that accounts for the 
        # differences in the z direction:
        #  multiply the whle thing by :   np.array([1,1,ZSTUFF]) 
        # where ZSTUFF are the terms associated with the z direction
        R = np.sqrt(posvec[0]**2 + posvec[1]**2)
        B = rd + np.sqrt(posvec[2]**2 + zd**2)
        Zstuff = B/(np.sqrt(posvec[2]**2+zd**2))
        Miyamoto = -self.G*M/(np.sqrt(R**2 + B**2)) * posvec * np.array([1,1,Zstuff])
        
        return Miyamoto
    
    def M31Accel(self, r): # input should include the position vector, r
        """input:
                    r = position vector
           return:
                    3D vector of total acceleration"""
            ### Call the previous functions for the halo, bulge and disk
        # **** these functions will take as inputs variable we defined in the initialization of the class like 
        #      self.rdisk etc. 
        accbulge = self.HernquistAccel(self.Mbulge, self.rbulge, r)
        acchalo = self.HernquistAccel(self.Mhalo, self.rhalo, r)
        accdisk = self.MiyamotoNagaiAccel(self.Mdisk, self.rdisk, r)
        acctotal = accbulge + acchalo + accdisk
            
        return acctotal
    
    def LeapFrog(self, dt, r, v):  # take as input r and v, which are VECTORS. Assume it is ONE vector at 
                                       # a time
        """inputs:
                    dt = change in time interval
                    r = position vector
                    v = velocity vector
           return:
                    New position and velocity vectors after time dt"""
        # predict the position at the next half timestep
        rhalf = r * (v*dt/2)
            
        # predict the final velocity at the next timestep using the acceleration field at the rhalf 
        # position 
        vnew = v + self.M31Accel(rhalf)*dt 
            
        # predict the final position using the average of the current velocity and the final velocity
        rnew = rhalf + vnew*dt/2
            
        return rnew,vnew
        
    def OrbitIntegrator(self, t0, dt, tmax):
        """inputs:
                    t0 = starting time
                    dt = change in time interval
                    tmax = final time
           return:
                    M33's fututre position relative to M31"""
        # initialize the time to the input starting time
        t = t0
        
        # initialize an empty array of size :  rows int(tmax/dt)+2  , columns 7
        orbit = np.zeros((int(tmax/dt)+2, 7))
            
        # initialize the first row of the orbit
        orbit[0] = t0, *tuple(self.r0), *tuple(self.v0)
        # this above is equivalent to 
        # orbit[0] = t0, self.r0[0], self.r0[1], self.r0[2], self.v0[0], self.v0[1], self.v0[2]
            
        # initialize a counter for the orbit.  
        i = 1 # since we already set the 0th values, we start the counter at 1
            
        # start the integration (advancing in time steps and computing LeapFrog at each step)
        while (t<tmax): # as long as t has not exceeded the maximal time
            # **** advance the time by one timestep, dt
            t+=dt
                
            # **** store the new time in the first element of the ith row
            orbit[i][0] = t
                
            # ***** advance the position and velocity using the LeapFrog scheme
            newpos, newvel = self.LeapFrog(dt, orbit[i-1][1:4], orbit[i-1][4:7])
                
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of 
            #       orbit
            orbit[i][1:4] = newpos
                
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of 
            #       orbit
            orbit[i][4:7] = newvel
                
            # **** update counter i , where i is keeping track of the number of rows (i.e. the number of 
            #      time steps)
            i += 1
                
            # write the data to a file
        np.savetxt(self.filename, orbit, fmt = "%11.3f"*7, comments='#', 
                   header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                   .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))

In [122]:
# Assigns class to a varible that can be called
M33Orb = M33AnalyticOrbit('M33.txt')

In [118]:
# perferm integration with start, end, and time incriments in GYR
M33Orb.OrbitIntegrator(0,0.5,10)

/home/jugaitafa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: overflow encountered in double_scalars
/home/jugaitafa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: invalid value encountered in double_scalars
/home/jugaitafa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:101: RuntimeWarning: overflow encountered in double_scalars
/home/jugaitafa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:102: RuntimeWarning: overflow encountered in double_scalars
/home/jugaitafa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:103: RuntimeWarning: overflow encountered in double_scalars
/home/jugaitafa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:103: RuntimeWarning: invalid value encountered in double_scalars


In [119]:
# creates a data array
data = np.genfromtxt('M33.txt',dtype=None, names=True)

In [120]:
# Orbit array from HW6
M31COM = np.genfromtxt('Orbit_M31.txt',comments='#',names=True)
M33COM = np.genfromtxt('Orbit_M33.txt',comments='#',names=True)

# Finds distance from M31 to M33
dist = np.sqrt(M33Orb['x']**2 + M33Orb['y']**2 + M33Orb['z']**2)
M33M31Dist = VectorDiff([M33COM['x'],M33COM['y'],M33COM['z']],[M31COM['x'],M31COM['y'],M31COM['z']])

# Plots both the analytic and the read in orbits
plt.plot(M33Orb['t'],dist,label='Analytic Orbit')
plt.plot(M33COM['t'],M33M31Dist,label='Simulation Orbit')
plt.xlabel('Time (Gyrs)')
plt.ylabel('Separation (kpc)')
plt.title('Distance between M33 and M31')
plt.legend(loc='upper right')

TypeError: 'M33AnalyticOrbit' object is not subscriptable

In [ ]:
# ANSWERS TO QUESTIONS
######################

# Q2) Referred to Sean Cunningham's, Joseph Hickey's, and Madison Walder's graphs:
#     Up until around 1 Gyr, both the analytic and the simulation data are nearly identical.  After 1Gyr, the 
#     two sets of data diverge significantly.  The simulation data exhibits sporadic changes in seperation and
#     velocity, whereas the analytic exhibits smooth curves for both.

# Q3) As stated in Q4, we are missing the MW and the phsical contributions that it would have on the M31-M33
#     system.

# Q4) If we were to include the MW in the calculations, the system would become a 3-body system.  Since 3-body
#     systems are very difficult to solve for, the best way to do this would be to change it as a 2-body system
#     by treating M33 and M31 as one massive body.  We can do this because M31 is so much more massive than M33
#     that doing this would still give us an accurate answer.